In [1]:
import html5lib
import lxml

from bs4 import BeautifulSoup as bs

import requests
import re

import pandas as pd

import os

import time
import random

In [2]:
years = range(2010, 2024)

In [3]:
team_to_url_df = pd.DataFrame(
    data=[
        [
            'crd', #Cardinals
            'atl', #Falcons
            'rav', #Ravens
            'buf', #Buffalos
            'car', #Panthers
            'chi', #Bears
            'cin', #Bengals
            'cle', #Browns
            'dal', #Cowboys
            'den', #Broncos
            'det', #Lions
            'gnb', #Packers
            'htx', #Texans
            'clt', # Colts
            'jax', #Jaguars
            'kan', #Chiefs
            'rai', #Raiders
            'sdg', #Chargers
            'ram', #Rams
            'mia', #Dolphins
            'min', #Vikings
            'nwe', #Patriots
            'nor', #Saints
            'nyg', #Giants
            'nyj', #Jets
            'phi', #Eagles
            'pit', #Steelers
            'sfo', #49ers
            'sea', #Seahawks
            'tam', #Buccaneers
            'oti', #Titans
            'was' #Commanders
        ]
    ], 
    columns=[
        'Cardinals',
        'Falcons',
        'Ravens',
        'Buffalos',
        'Panthers',
        'Bears',
        'Bengals',
        'Browns',
        'Cowboys',
        'Broncos',
        'Lions',
        'Packers',
        'Texans',
        'Colts',
        'Jaguars',
        'Chiefs',
        'Raiders',
        'Chargers',
        'Rams',
        'Dolphins',
        'Vikings',
        'Patriots',
        'Saints',
        'Giants',
        'Jets',
        'Eagles',
        'Steelers',
        '49ers',
        'Seahawks',
        'Buccaneers',
        'Titans',
        'Commanders'
    ])

# Import Roster 2010-2023

In [4]:
# use this function before creating tables
def roster_df(url:str):
    
    html_roster_str = requests.get(url).text

    soup = bs(html_roster_str, 'lxml')

    html_roster = soup.find_all('div', class_='table_wrapper')[1]
    
        # index the 5th element to get table
    html_roster_children_li = []
    for hrc in html_roster.children:
        html_roster_children_li.append(hrc)

    df = pd.read_html(html_roster_children_li[4])[0]

    time.sleep(random.randrange(5,8))

    return df

In [5]:
def create_roster_csv(team_url:str, year:int):

    # format team_to_url_df list to choose team, the second bracket is for the year
    roster_url = 'http://www.pro-football-reference.com/teams/{}/{}_roster.htm'
            
    #webscrape roster dataframe
    url = roster_url.format(team_url, year)
    df = roster_df(url=url)
    
    #drop last col
    df = df.drop(columns=['Drafted (tm/rnd/yr)'], axis=1)

    try:
        df = df.drop(columns=['Salary'], axis=1)
    except:
        pass
                         
    return df

In [13]:
def mani_roster(roster_df:pd)->pd:

    #rename columns
    roster_df.columns = ['No.', 'Player', 'Age', 'Position', 'Games_Played_in_Season', 
                  'Games_Started_in_Season', 'Weight', 'Height', 'College/University', 'BirthDate',
                 'NFL_Years', 'Player_Value']

    #change Rook to 0 in yrs column
    roster_df['NFL_Years'] = roster_df['NFL_Years'].replace('Rook', 0)

    #change datataype to integer
    df['NFL_Years'] = df['NFL_Years'].astype('float64')

    roster_df['Team'] = [team] * len(roster_df)
    roster_df['Year'] = [year] * len(roster_df)

    #Reorder clumns
    roster_df = roster_df[['No.', 'Player', 'BirthDate', 'Age', 'Height', 'Weight', 
             'College/University', 'Position', 'Games_Played_in_Season',
            'Games_Started_in_Season', 'NFL_Years', 'Player_Value', 'Team', 'Year']]

    return roster_df

In [14]:
for year in years:
    for team in team_to_url_df.columns:

        team_url = team_to_url_df[team].values[0]
        
        df = create_roster_csv(team_url=team_url, year=year)

        df = mani_roster(roster_df=df)
        
        #where to save datafram
        df.to_csv(f'NFL_Data_{year}\{team}\{team}_roster')
        
        print(df)

   

C:\Users\Family\AppData\Local\Temp\ipykernel_5448\3057426242.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html_roster_children_li[4])[0]


     No.             Player   BirthDate   Age Height  Weight  \
0   21.0     Hamza Abdullah   8/20/1983  27.0    6-2   216.0   
1   27.0      Michael Adams   6/17/1985  25.0    5-8   178.0   
2    3.0     Derek Anderson   6/15/1983  27.0    6-6   235.0   
3    2.0     Richard Bartel    2/3/1983  27.0    6-3   246.0   
4   74.0  D'Anthony Batiste   3/29/1982  28.0    6-4   314.0   
..   ...                ...         ...   ...    ...     ...   
56  92.0       Dan Williams    6/1/1987  23.0    6-2   330.0   
57  14.0   Stephen Williams   6/29/1986  24.0    6-5   199.0   
58  24.0      Adrian Wilson  10/12/1979  31.0    6-3   230.0   
59  31.0       Jason Wright   7/12/1982  28.0   5-10   212.0   
60   NaN         Team Total         NaN  26.7  6-2.0   245.4   

    College/University Position  Games_Played_in_Season  \
0       Washington St.       DB                       6   
1            Louisiana       DB                      16   
2           Oregon St.       QB                      1

C:\Users\Family\AppData\Local\Temp\ipykernel_5448\3057426242.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html_roster_children_li[4])[0]


     No.              Player   BirthDate   Age Height  Weight  \
0   55.0        John Abraham    5/6/1978  32.0    6-4   263.0   
1   59.0      Spencer Adkins   5/16/1987  23.0   5-11   246.0   
2   98.0     Jamaal Anderson    2/6/1986  24.0    6-5   288.0   
3   95.0  Jonathan Babineaux  10/12/1981  29.0    6-2   300.0   
4   72.0           Sam Baker   5/30/1985  25.0    6-5   308.0   
5   71.0       Kroy Biermann   9/12/1985  25.0    6-3   242.0   
6   63.0      Justin Blalock  12/20/1983  27.0    6-3   320.0   
7   67.0       Rob Bruggeman   3/21/1986  24.0    6-4   290.0   
8    3.0         Matt Bryant   5/29/1975  35.0    5-9   203.0   
9   77.0         Tyson Clabo  10/17/1981  29.0    6-6   315.0   
10  26.0        Erik Coleman    5/6/1982  28.0   5-10   207.0   
11  73.0         Harvey Dahl   6/24/1981  29.0    6-5   308.0   
12  92.0      Chauncey Davis   1/27/1983  27.0    6-2   262.0   
13  28.0       Thomas DeCoud   3/19/1985  25.0    6-1   196.0   
14  83.0       Harry Doug

C:\Users\Family\AppData\Local\Temp\ipykernel_5448\3057426242.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html_roster_children_li[4])[0]


...

     No.                Player   BirthDate   Age Height  Weight  \
0    7.0       Shaquil Barrett  11/17/1992  31.0    6-2   250.0   
1   52.0            K.J. Britt    6/4/1999  24.0    6-0   235.0   
2    5.0          Jake Camarda   4/20/1999  24.0    6-1   193.0   
3   54.0         Lavonte David   1/23/1990  33.0    6-1   233.0   
4   24.0         Carlton Davis  12/31/1996  27.0    6-1   206.0   
..   ...                   ...         ...   ...    ...     ...   
56  45.0           Devin White   2/17/1998  25.0    6-0   237.0   
57   1.0         Rachaad White   1/22/1999  24.0    6-0   214.0   
58  31.0  Antoine Winfield Jr.   8/16/1998  25.0    5-9   203.0   
59  78.0         Tristan Wirfs   1/24/1999  24.0    6-5   320.0   
60   NaN            Team Total         NaN  25.8  6-2.0   242.9   

                                   College/University Position  \
0                                        Colorado St.     LOLB   
1                                              Auburn       LB 

C:\Users\Family\AppData\Local\Temp\ipykernel_5448\3057426242.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html_roster_children_li[4])[0]


     No.                 Player  BirthDate   Age Height  Weight  \
0    2.0        Azeez Al-Shaair   8/4/1997  26.0    6-2   228.0   
1   96.0           Denico Autry  7/15/1990  33.0    6-5   285.0   
2   23.0              Tre Avery  2/26/1997  26.0   5-11   181.0   
3   90.0         Ross Blacklock   7/9/1998  25.0    6-3   290.0   
4   97.0        Quinton Bohanna  3/16/1999  24.0    6-4   338.0   
..   ...                    ...        ...   ...    ...     ...   
79  15.0  Nick Westbrook-Ikhine  3/21/1997  26.0    6-2   211.0   
80  81.0             Josh Whyle   9/8/1999  24.0    6-6   248.0   
81   7.0           Malik Willis  5/25/1999  24.0    6-1   219.0   
82  13.0             Ty Zentner   6/7/1998  25.0    6-2   199.0   
83   NaN             Team Total        NaN  26.2  6-2.0   246.7   

                     College/University Position  Games_Played_in_Season  \
0                      Florida Atlantic     LILB                      17   
1   East Mississippi CC,Mississippi St.    

C:\Users\Family\AppData\Local\Temp\ipykernel_5448\3057426242.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html_roster_children_li[4])[0]


     No.             Player   BirthDate   Age Height  Weight  \
0   58.0   Tucker Addington   7/16/1997  26.0    6-2   230.0   
1   62.0     Alex Akingbulu  12/28/1997  26.0    6-5   300.0   
2   93.0     Jonathan Allen   1/16/1995  28.0    6-3   300.0   
3   92.0  Abdullah Anderson   1/24/1996  27.0    6-4   295.0   
4   40.0         Alex Armah   5/17/1994  29.0    6-1   255.0   
..   ...                ...         ...   ...    ...     ...   
67   0.0     Jace Whittaker   7/16/1995  28.0   5-11   185.0   
68  18.0  Jonathan Williams    2/2/1994  29.0    6-0   217.0   
69  71.0       Andrew Wylie   8/19/1994  29.0    6-6   309.0   
70  99.0        Chase Young   4/14/1999  24.0    6-5   264.0   
71   NaN         Team Total         NaN  26.3  6-2.3   244.5   

   College/University Position  Games_Played_in_Season  \
0     Sam Houston St.       LS                       3   
1     UCLA,Fresno St.        T                       2   
2             Alabama       DT                      16   

In [ ]:
stop

In [ ]:
roster_team_year_li = create_roster_csv(years=years)

In [ ]:
roster_team_year_li

# 2023 Rosters

In [ ]:
# use this function before creating tables
def roster_df_2023(url:str):
    
    html = requests.get(url).text
    
    soup = bs(html, 'html.parser')

    table = soup.find_all(class_='table_wrapper')[0]

    str_table = str(table)

    start_num_str_comment_tag = str_table.find('<!--')+4

    table_html = bs(str_table[start_num_str_comment_tag:], 'html.parser')

    df = pd.read_html(str(table_html), index_col=1)[0]
    
    time.sleep(random.randrange(5,8))
    return df

In [ ]:
def create_roster_csv(years:None):
    
    years = years
    
    team_to_url_df = pd.DataFrame(
        data=[
            [
                'crd', #Cardinals
                'atl', #Falcons
                'rav', #Ravens
                'buf', #Buffalos
                'car', #Panthers
                'chi', #Bears
                'cin', #Bengals
                'cle', #Browns
                'dal', #Cowboys
                'den', #Broncos
                'det', #Lions
                'gnb', #Packers
                'htx', #Texans
                'clt', # Colts
                'jax', #Jaguars
                'kan', #Chiefs
                'rai', #Raiders
                'sdg', #Chargers
                'ram', #Rams
                'mia', #Dolphins
                'min', #Vikings
                'nwe', #Patriots
                'nor', #Saints
                'nyg', #Giants
                'nyj', #Jets
                'phi', #Eagles
                'pit', #Steelers
                'sfo', #49ers
                'sea', #Seahawks
                'tam', #Buccaneers
                'oti', #Titans
                'was' #Commanders
            ]
        ], 
        columns=[
            'Cardinals',
            'Falcons',
            'Ravens',
            'Buffalos',
            'Panthers',
            'Bears',
            'Bengals',
            'Browns',
            'Cowboys',
            'Broncos',
            'Lions',
            'Packers',
            'Texans',
            'Colts',
            'Jaguars',
            'Chiefs',
            'Raiders',
            'Chargers',
            'Rams',
            'Dolphins',
            'Vikings',
            'Patriots',
            'Saints',
            'Giants',
            'Jets',
            'Eagles',
            'Steelers',
            '49ers',
            'Seahawks',
            'Buccaneers',
            'Titans',
            'Commanders'
        ])
    # format team_to_url_df list to choose team, the second bracket is for the year
    roster_url = 'http://www.pro-football-reference.com/teams/{}/{}_roster.htm'
    
    #to observe imported df
    roster_team_year_li = []
    
    for year in years:
        for team_url in team_to_url_df:
            
            #webscrape roster dataframe
            url = roster_url.format(team_to_url_df[team_url].values[0], year)
            df = roster_df_2023(url=url)
            
            #drop last col
            df.drop(columns=['Drafted (tm/rnd/yr)'], axis=1)
                        
            df.insert(1, 'Team', team_url)
            
            #where to save datafram
            df.to_csv('NFL_Data_{}\\'.format(year) + team_url + '\\{}_roster'.format(team_url))
           
            #to observe imported df
            roster_team_year_li.append(df)
            
    return roster_team_year_li

In [ ]:
roster_team_year_li = create_roster_csv(years=[2023])